In [12]:
import pandas as pd
import numpy as np

In [44]:
seasontickets = pd.read_csv('seasontickets.csv')
seasontickets = seasontickets[seasontickets['Status'] == 'Active']
seasontickets['User Id'] = seasontickets['User Id'].astype(int)
seasontickets = seasontickets.drop_duplicates(subset=['User Id'])

tickets = pd.read_csv('games.csv')
tickets = tickets.dropna(subset=['User Id']) # without cashier tickets
tickets['User Id'] = tickets['User Id'].astype(int)
tickets = tickets[tickets['Status'] == 'Active']

In [45]:
print('seasontickets shape:', seasontickets.shape)
print('tickets shape:', tickets.shape)

seasontickets shape: (3421, 54)
tickets shape: (4111, 54)


In [46]:
without_members = tickets[~tickets['User Id'].isin(seasontickets['User Id'])]
print('Tickets bought by non-members:', without_members.shape[0])
without_members

Tickets bought by non-members: 2305


,Product Id,Product,Status,Id,Fan / Company,User Id,assign using ID number,First name,Last name,Email,...,Delivery type,Ticket Note,Transaction Note,Role,ReturnRequestStatus,Voucher number,Voucher name,Voucher value,BindingTime,Unnamed: 53
8,4270,גביע ווינר שלב הבתים #1: מכבי ראשון לציון 🏠,Active,3449096,עידן מזרחי - מכבי ראשון לציון,1429510,300606043,עידן,מזרחי - מכבי ראשון לציון,Idan.mizrachi.99@gmail.com,...,Online ticket,NaN,NaN,Administrator,NaN,NaN,NaN,NaN,NaN,NaN
57,4270,גביע ווינר שלב הבתים #1: מכבי ראשון לציון 🏠,Active,3448919,אורטל לוי,1435228,066406752,אורטל,לוי,ortalm18@gmail.com,...,Online ticket,NaN,NaN,User,NaN,NaN,NaN,NaN,NaN,NaN
60,4270,גביע ווינר שלב הבתים #1: מכבי ראשון לציון 🏠,Active,3448904,Marc Miller,1435227,A23058789,Marc,Miller,mmiller@tocohillrealty.com,...,Online ticket,NaN,NaN,User,NaN,יאללה אפרתה,יאללה בתי ספר,NaN,NaN,NaN
61,4270,גביע ווינר שלב הבתים #1: מכבי ראשון לציון 🏠,Active,3448901,עילאי גולדברג,29060,222745820,עילאי,גולדברג,yaelgold@gmail.com,...,Online ticket,NaN,NaN,User,NaN,NaN,NaN,NaN,NaN,NaN
62,4270,גביע ווינר שלב הבתים #1: מכבי ראשון לציון 🏠,Active,3448896,שרון נדב,1435226,036107399,שרון,נדב,sharon_nadav@walla.co.il,...,Online ticket,NaN,NaN,User,NaN,יאללה גוננים,יאללה בתי ספר,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4572,4633,חצי גמר גביע ווינר: מכבי תל אביב,Active,3463102,גיא גוטמן,1390682,031506249,גיא,גוטמן,guy.igfs@gmail.com,...,Online ticket,NaN,NaN,Administrator,NaN,NaN,NaN,NaN,NaN,NaN
4573,4633,חצי גמר גביע ווינר: מכבי תל אביב,Active,3463108,אלעזר בנג'מין,46092,33301383,אלעזר,בנג'מין,elazar.m.benjamin@gmail.com,...,Online ticket,NaN,NaN,Administrator,NaN,NaN,NaN,NaN,NaN,NaN
4574,4633,חצי גמר גביע ווינר: מכבי תל אביב,Active,3463107,גיא גוטמן,1390682,031506249,גיא,גוטמן,guy.igfs@gmail.com,...,Online ticket,NaN,NaN,Administrator,NaN,NaN,NaN,NaN,NaN,NaN
4575,4633,חצי גמר גביע ווינר: מכבי תל אביב,Active,3463101,גיא גוטמן,1390682,031506249,גיא,גוטמן,guy.igfs@gmail.com,...,Online ticket,NaN,NaN,Administrator,NaN,NaN,NaN,NaN,NaN,NaN


In [47]:
# --- helpers ---
def concat_unique(s):
    s = s.dropna().astype(str).str.strip()
    if s.empty:
        return np.nan
    return ', '.join(pd.unique(s))

def first_nonnull(s):
    s = s.dropna()
    return s.iloc[0] if not s.empty else np.nan

def most_common_or_join(s):
    s = s.dropna().astype(str).str.strip()
    if s.empty:
        return np.nan
    m = s.mode()
    # if one clear mode, use it; otherwise join uniques
    return m.iloc[0] if len(m) == 1 else ', '.join(pd.unique(s))

# --- tidy up & casting ---
df = without_members.copy()

# normalize name whitespace to avoid duplicate keys
df['Fan / Company'] = df['Fan / Company'].astype(str).str.strip()

# make sure Price is numeric
# (keeps digits and dot; strips currency symbols, commas, spaces)
df['Price'] = (
    df['Price']
      .astype(str)
      .str.replace(r'[^0-9.\-]', '', regex=True)
      .replace({'': np.nan, '-': np.nan})
      .astype(float)
)

# ensure Birth date is datetime
df['Birth date'] = pd.to_datetime(df['Birth date'], errors='coerce')

# --- 1) one line per (person, product) ---
per_product = (
    df.groupby(['Fan / Company', 'User Id', 'Product'], dropna=False)
      .agg(
          tickets_bought = ('Product', 'size'),
          total_price    = ('Price', 'sum'),
          ticket_price_types = ('Ticket price types', concat_unique),
          voucher_numbers    = ('Voucher number', concat_unique),
          voucher_names      = ('Voucher name', concat_unique),
          phone              = ('Phone', most_common_or_join),
          email              = ('Email', most_common_or_join),
          birth_date         = ('Birth date', first_nonnull)
      )
      .reset_index()
)

# --- 2) one line per person ---
person_summary = (
    df.groupby(['Fan / Company', 'User Id'], dropna=False)
      .agg(
          phone            = ('Phone', most_common_or_join),
          email            = ('Email', most_common_or_join),
          total_tickets    = ('Product', 'size'),
          product_list     = ('Product', concat_unique),
          distinct_products= ('Product', 'nunique'),
          total_price      = ('Price', 'sum'),
          ticket_price_types = ('Ticket price types', concat_unique),
          voucher_numbers    = ('Voucher number', concat_unique),
          voucher_names      = ('Voucher name', concat_unique),
          birth_date         = ('Birth date', first_nonnull)
      )
      .reset_index()
)

today = pd.Timestamp.today().normalize()
bd = person_summary['birth_date']

# has the person already had their birthday this year?
has_had_bday = (
    (bd.dt.month < today.month) |
    ((bd.dt.month == today.month) & (bd.dt.day <= today.day))
)

age = (today.year - bd.dt.year) - (~has_had_bday).astype(int)

# keep NaN where birth_date is missing
person_summary['age'] = age.where(bd.notna())

# optional: order columns nicely
person_summary = person_summary[
    ['Fan / Company', 'User Id', 'phone', 'email',
     'total_tickets', 'distinct_products', 'product_list',
     'total_price', 'ticket_price_types', 'voucher_numbers', 'voucher_names', 'age']
]

person_summary

,Fan / Company,User Id,phone,email,total_tickets,distinct_products,product_list,total_price,ticket_price_types,voucher_numbers,voucher_names,age
0,Adam Berrous,1420654,0552864391,adamberrous@gmail.com,1,1,גביע ווינר שלב הבתים #1: מכבי ראשון לציון 🏠,50.0,Child,NaN,NaN,12.0
1,Alejandra Rosenberg,1435692,0515690960,alerosenberg81@hotmail.com,4,1,גביע ווינר שלב הבתים #2: מכבי עירוני רמת גן 🏠,440.0,Adult,NaN,NaN,44.0
2,Anthony Lamb,1435129,NaN,NaN,2,1,גביע ווינר שלב הבתים #1: מכבי ראשון לציון 🏠,0.0,Complementary,NaN,NaN,NaN
3,Asaf Ataria,1424949,0503744476,asaf.ataria@gmail.com,1,1,חצי גמר גביע ווינר: מכבי תל אביב,75.0,Adult,NaN,NaN,17.0
4,Austin Wiley,1349202,NaN,Austin.wiley50@gmail.com,1,1,חצי גמר גביע ווינר: מכבי תל אביב,0.0,Complementary,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
995,תמר בנאי,1404140,NaN,NaN,1,1,חצי גמר גביע ווינר: מכבי תל אביב,75.0,Adult,NaN,NaN,12.0
996,תמר זקנשטיין,1435263,0524492550,tamarst4@gmail.com,6,1,גביע ווינר שלב הבתים #2: מכבי עירוני רמת גן 🏠,0.0,Adult,יאללה יפה נוף,יאללה בתי ספר,39.0
997,תמר שגיב,1434826,NaN,NaN,1,1,גביע ווינר שלב הבתים #2: מכבי עירוני רמת גן 🏠,0.0,Adult,יאללה אפרתה,יאללה בתי ספר,16.0
998,תמר שכטר,1343401,NaN,NaN,1,1,גביע ווינר שלב הבתים #2: מכבי עירוני רמת גן 🏠,0.0,Child,יאללה גוננים,יאללה בתי ספר,10.0


In [48]:
person_summary.to_excel('winnercup_tickets_non_members.xlsx', index=False)